In [1]:
#set cwd to /PiperNET
import os
os.chdir('..')

import subprocess
from pathlib import Path
import pandas as pd
import argparse
from Bio import SeqIO
from io import StringIO
from src.rnaseq_utils import get_config
from src.rnaseq_utils import download_enzymeDB

In [6]:
#download enzymeDB
enzymeDB_path = 'data/rna-seq/enzymeDB.csv'
enzyme_seq_path = 'data/rna-seq/enzyme_seq'
download_enzymeDB(enzymeDB_path, enzyme_seq_path)


#load sample folder paths from config.yaml
config_path = Path('config.yaml')
blastDB_paths = get_config(config_path, data='blast', db_dirs='blastDB')
proteome_paths = get_config(config_path, data='rna-seq', format='proteomes.csv')


#import proteomes
proteome_list = {}
for sample, path in proteome_paths.items():
    try:
        proteome_list[sample] = pd.read_csv(path, index_col='id')
    except FileNotFoundError:
        print(f'{path} not found for sample {sample}. Skipping to the next file.')

'enzyme_seq' folder already exists at data/rna-seq/enzyme_seq
AtPAL2 sequence already present at 'data/rna-seq/enzyme_seq/AtPAL2.fasta'
PaPAL sequence already present at 'data/rna-seq/enzyme_seq/PaPAL.fasta'
AtC4H sequence already present at 'data/rna-seq/enzyme_seq/AtC4H.fasta'
MsC4H sequence already present at 'data/rna-seq/enzyme_seq/MsC4H.fasta'
At4CL1 sequence already present at 'data/rna-seq/enzyme_seq/At4CL1.fasta'
At4CL2 sequence already present at 'data/rna-seq/enzyme_seq/At4CL2.fasta'
At4CL3 sequence already present at 'data/rna-seq/enzyme_seq/At4CL3.fasta'
At4CL4 sequence already present at 'data/rna-seq/enzyme_seq/At4CL4.fasta'
Nt4CL1 sequence already present at 'data/rna-seq/enzyme_seq/Nt4CL1.fasta'
Nt4CL2 sequence already present at 'data/rna-seq/enzyme_seq/Nt4CL2.fasta'
HlCL1 sequence already present at 'data/rna-seq/enzyme_seq/HlCL1.fasta'
PhCL1 sequence already present at 'data/rna-seq/enzyme_seq/PhCL1.fasta'
NpCL1 sequence already present at 'data/rna-seq/enzyme_seq/N

In [44]:
########## BLASTp search ##########

def run_blastp(seq_paths, blastDB_paths, min_cov = 0, min_sim = 0):
    
    '''To document'''
    
    hits = pd.DataFrame()  # empty DataFrame
    
    for query in seq_paths.values():
        for sample, blastDB in zip(blastDB_paths.keys(), blastDB_paths.values()):
            print(f'Querying {query} in {sample} transcriptome...')

            #construct command
            command = f'blastp -db {blastDB} -query {query} -evalue 1e-50 -outfmt "10 delim=, qacc sseqid qcovs score evalue pident ppos sseq"'

            #run command
            output = subprocess.run(command, shell=True, capture_output=True, text=True)

            #convert to dataframe
            if output.returncode == 0:
                output = output.stdout
                output = pd.read_csv(StringIO(output), header=None, names=['query', 'hit', 'coverage', 'score', 'evalue', 'identity', 'similarity', 'sequence'])

                #create full_id column
                output['blast_id'] = output.apply(lambda row: f"{row['hit']}_cov{row['coverage']}_sim{round(row['similarity'])}", axis=1)
                
                #print number of retrieved hits
                print(f"{output.shape[0]} hits found for {query.stem} in {sample} transcriptome.")

                #filter by coverage and similarity
                filt = (output['coverage'] > min_cov) & (output['similarity'] > min_sim)
                output = output.loc[filt]

                #append output to blast_results
                hits = pd.concat([hits, output], ignore_index=True)

            else:
                print(f"An error occurred while running the BLASTp command!")

    #remove duplicates
    hits['query'] = hits['query'].apply(lambda x: [x]) #convert to list
    grouped = hits.groupby('hit').agg({'query': 'sum'})
    hits = hits.drop(columns='query')
    hits = pd.merge(hits, grouped, left_on='hit', right_index=True)
    hits['query'] = hits['query'].apply(set) #convert to set

    return hits

In [45]:
#PKS
enzyme_class = 'PKS'
seq_paths = get_config(config_path, data='blast', enzymeDB='enzyme_seq', enzyme_class=enzyme_class)
pks = run_blastp(seq_paths, blastDB_paths, min_cov=50, min_sim=50)

Querying data/rna-seq/enzyme_seq/PmSPS1.fasta in piper09 transcriptome...
14 hits found for PmSPS1 in piper09 transcriptome.
Querying data/rna-seq/enzyme_seq/PmSPS1.fasta in piper12 transcriptome...
8 hits found for PmSPS1 in piper12 transcriptome.
Querying data/rna-seq/enzyme_seq/PmSPS1.fasta in piper56 transcriptome...
14 hits found for PmSPS1 in piper56 transcriptome.
Querying data/rna-seq/enzyme_seq/VvSTS1.fasta in piper09 transcriptome...
14 hits found for VvSTS1 in piper09 transcriptome.
Querying data/rna-seq/enzyme_seq/VvSTS1.fasta in piper12 transcriptome...
7 hits found for VvSTS1 in piper12 transcriptome.
Querying data/rna-seq/enzyme_seq/VvSTS1.fasta in piper56 transcriptome...
14 hits found for VvSTS1 in piper56 transcriptome.
Querying data/rna-seq/enzyme_seq/ClCURS1.fasta in piper09 transcriptome...
13 hits found for ClCURS1 in piper09 transcriptome.
Querying data/rna-seq/enzyme_seq/ClCURS1.fasta in piper12 transcriptome...
7 hits found for ClCURS1 in piper12 transcriptome.

In [46]:
pks

,hit,coverage,score,evalue,identity,similarity,sequence,blast_id,query
0,Pfim_g13608_i1.p1,99,1763,0.000000e+00,83.038,90.38,MSKTVEEIQAAQRARGPATVLAIGTATPANVVYQADYPDYYFRITK...,Pfim_g13608_i1.p1_cov99_sim90,"{PmSPS1, VvSTS1, ClCURS1}"
31,Pfim_g13608_i1.p1,98,1615,0.000000e+00,74.870,88.86,TVEEIQAAQRARGPATVLAIGTATPANVVYQADYPDYYFRITKSEH...,Pfim_g13608_i1.p1_cov98_sim89,"{PmSPS1, VvSTS1, ClCURS1}"
59,Pfim_g13608_i1.p1,98,1282,8.710000e-177,57.180,77.55,IQAAQRARGPATVLAIGTATPANVVYQADYPDYYFRITKSEHMTEL...,Pfim_g13608_i1.p1_cov98_sim78,"{PmSPS1, VvSTS1, ClCURS1}"
1,Pfim_g19684_i0.p1,97,1686,0.000000e+00,80.879,89.66,TVDDIRKAQRAEGPATVLAIGTATPANCVYQDDYPDYYFRITNSEH...,Pfim_g19684_i0.p1_cov97_sim90,"{PmSPS1, VvSTS1, ClCURS1}"
30,Pfim_g19684_i0.p1,99,1641,0.000000e+00,76.093,88.17,MATVDDIRKAQRAEGPATVLAIGTATPANCVYQDDYPDYYFRITNS...,Pfim_g19684_i0.p1_cov99_sim88,"{PmSPS1, VvSTS1, ClCURS1}"
...,...,...,...,...,...,...,...,...,...
27,Pnig_g20709_i0.p1,52,794,5.740000e-105,74.146,81.95,NRYFHQTEELLEKNLNICAYMAPSLDARQDIAVEEVPKLAKEAATT...,Pnig_g20709_i0.p1_cov52_sim82,"{PmSPS1, VvSTS1, ClCURS1}"
54,Pnig_g20709_i0.p1,51,734,4.510000e-96,67.000,82.00,RYFHQTEELLEKNLNICAYMAPSLDARQDIAVEEVPKLAKEAATTA...,Pnig_g20709_i0.p1_cov51_sim82,"{PmSPS1, VvSTS1, ClCURS1}"
81,Pnig_g20709_i0.p1,52,592,1.320000e-74,52.217,70.44,NRYFHQTEELLEKNLNICAYMAPSLDARQDIAVEEVPKLAKEAATT...,Pnig_g20709_i0.p1_cov52_sim70,"{PmSPS1, VvSTS1, ClCURS1}"
28,Pnig_g44643_i0.p1,56,449,2.660000e-53,42.411,61.61,LCKTTTVKTRYVVMSEEILNKYPELAVEGLPTVKQRLEVCNDAVTQ...,Pnig_g44643_i0.p1_cov56_sim62,"{PmSPS1, VvSTS1}"


In [48]:

#retrieve hits from transcriptomes
data = pd.DataFrame()  # empty DataFrame

for sample, proteome in zip(proteome_list.keys(), proteome_list.values()):
    hits = proteome.loc[proteome.index.isin(pks['hit'])]

    #append output to blast_results
    data = pd.concat([data, hits])

In [60]:
# (~data['pfam'].str.contains('PF00195.23'))

data.loc['Pfim_g20283_i0.p1']

sequence               MLVHGSLFRCTPKDHGRGGDQAGHGSRREGHQGMGAARIQDYPPRL...
length                                                               310
orf_type                                                        complete
orientation                                                          (-)
orf_score                                                          35.89
transcript_length                                                   1333
transcript_coverage                                            11.983333
pfam                                                          PF02797.19
domain                 Chalcone and stilbene synthases, C-terminal do...
pfam_e-value                                                         0.0
blastp                 sp|A0A2Z5QL08|CHS_RHODA Chalcone synthase OS=R...
blastp_e-value                                                       0.0
signalp                                                            OTHER
targetp                                            

In [ ]:
# Usage: python blastp.py -db <path_to_blast_database> -q <path_to_query_sequence> --mcov 70 --msim 50 --proteome <path_to_proteome> -o <output_fasta>
#   -db, --blastDB: Path to the BLAST database.
#   -q, --query: Path to the query sequence (FASTA file).
#   --proteome: Path to the proteome sequence database (FASTA file).
#   --mcov: Minimum coverage for a BLAST hit to be retained (default: 0).
#   --msim: Minimum similarity for a BLAST hit to be retained (default: 0).
#   -o, --output: Output folder where matching sequences will be stored in a FASTA file.

def main():
    parser = argparse.ArgumentParser(description="XXX")
    parser.add_argument("-db", "--blastDB", dest="blastDB", required=True, help="Path to BLAST database.")    
    parser.add_argument("-q", "--query", dest="query", required=True, help="Path to query serquence (FASTA file).")
    parser.add_argument("--proteome", dest="proteome", required=True, help="Path to proteome (FASTA file).")
    parser.add_argument("--mcov", dest="min_cov", default=0, help="Minimum coverage for a BLAST hit to be retained.")
    parser.add_argument("--msim", dest="min_sim", default=0, help="Minimum similarity for a BLAST hit to be retained.")
    parser.add_argument("-o", "--output", dest="output_fasta", required=True, help="Output folder")
    args = parser.parse_args()

    #run blastp
    blast_results = blastp(args.blastDB, args.query, min_cov = float(args.min_cov), min_sim = float(args.min_sim))
    print(blast_results)
    hit_ids = set(blast_results['seq_id'])

    #empty list to store the matching sequences
    hit_sequences = []

    #retreive hit sequences from proteome.pep
    with open(args.proteome, 'r') as proteome_handle:
        for record in SeqIO.parse(proteome_handle, 'fasta'):
            if record.id in hit_ids:
                hit_sequences.append(record)

    # Write the matching sequences to the output FASTA file
    with open(args.output_fasta, 'w') as output_handle:
        for record in hit_sequences:
            full_id = blast_results.loc[blast_results['seq_id'] == record.id, 'full_id'].values[0]
            print(full_id)
            record.id = full_id
            record.description = ""  #clear record description field
            SeqIO.write(record, output_handle, 'fasta-2line')
            

    print(f"{len(hit_sequences)} BLAST hits stored to '{args.output_fasta}'.")


if __name__ == "__main__":
    main()